# Tensorflow Programming Guide

### 개관  
이 문서는 ML modeler 상에서 딥러닝 학습을 위한 예시와 함께 코드 작성을 위한 가이드라인을 제공합니다.


### 예시 코드 - MNIST
딥러닝 예제로 자주 사용되는 필기체 데이터를 이용한 예시입니다.    

0~9의 필기체 데이터를 학습하여 입력받은 필기체가 무슨 숫자인지 예측합니다.

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.python.estimator.export.export_output import PredictOutput



class Model:
    ################# 1 ######################
    def __init__(self,params):
        self.INPUT_TENSOR_NAME = "x"
        self.OUTPUT_TENSOR_NAME = "y"
        self.SIGNATURE_NAME = "serving_default"
        self.lr = params["learning_rate"]
        self.batch_size = params["batch_size"]  
        self.graph_params = ["loss", "accuracy"]

    ################# 2 ######################
    def model_fn(self,features, labels, mode, params):   
        first_hidden_layer = tf.layers.dense(tf.cast(features[self.INPUT_TENSOR_NAME], dtype=tf.float32), 50, activation=tf.nn.relu)
        second_hidden_layer = tf.layers.dense(first_hidden_layer, 100, activation=tf.nn.relu)
        last_hidden_layer = tf.layers.dense(second_hidden_layer, 50, activation=tf.nn.relu)
        output_layer = tf.layers.dense(last_hidden_layer, 10)
        predictions = tf.argmax(output_layer, 1)
    
    
        if mode == tf.estimator.ModeKeys.PREDICT:
            return tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={self.OUTPUT_TENSOR_NAME: predictions},
                export_outputs={self.SIGNATURE_NAME: PredictOutput({self.OUTPUT_TENSOR_NAME: predictions})}
            )
            
        y_label = tf.argmax(labels, 1)
        
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=output_layer))
        optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
    
        eval_metric_ops={}
        accuracy = tf.metrics.accuracy(y_label, predictions)
        eval_metric_ops["accuracy"] = accuracy
    
        return tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=eval_metric_ops) 
        
        
    ################# 3 ######################        
    def data_fn(self):
        data_path = "/mnt/project/demo/data/mnist-csv-data/data"
        trainX = np.loadtxt(data_path+"/trainX.csv")
        trainY = np.loadtxt(data_path+"/trainY.csv")
        evalX = np.loadtxt(data_path+"/evalX.csv")
        evalY = np.loadtxt(data_path+"/evalY.csv")
        self.trainX = trainX
        self.trainY = trainY
        self.evalX = evalX
        self.evalY =evalY
        
        
        
        
################# 4 ######################        
def preprocessing(image):
    import PIL.Image as pilimg
    img = pilimg.open(image)
    if img.size != (28,28):
        img = img.resize((28,28))
    if len(np.array(img).shape) == 3:
        img = img.convert('L')
    img = np.array(img).flatten()
    img = np.array([img])

    return img


def post_processing(output):
    if output in [0,2,4,6,8]:
        result = "짝수"
    elif output in [1,3,5,7,9]:
        result = "홀수"
   
    return result
   

    

################# 5 ######################
if __name__ == "__main__":

    params = {"learning_rate":0.001, "batch_size":100}

    user_model = Model(params)
    user_model.data_fn()
    model = tf.estimator.Estimator(model_fn=user_model.model_fn, model_dir=None, params=params)
    train_dic = {}
    if type(user_model.INPUT_TENSOR_NAME) != list:
        train_dic[user_model.INPUT_TENSOR_NAME] = user_model.trainX
    else:
        for i in range(len(user_model.INPUT_TENSOR_NAME)):
            train_dic[user_model.INPUT_TENSOR_NAME[i]] = user_model.trainX[i]
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
            x=train_dic,
            y=user_model.trainY,
            batch_size=user_model.batch_size,
            num_epochs=None,
            shuffle=True)  
    model.train(input_fn=lambda : train_input_fn(), steps=1000)

### 기본 형식    

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.python.estimator.export.export_output import PredictOutput

class Model:
    def __init__(self,params):
        self.SIGNATURE_NAME = "serving_default"
        self.INPUT_TENSOR_NAME = 
        self.OUTPUT_TENSOR_NAME = 
        self.lr =
        self.batch_size =
        self.graph_params = []
    
    
    def model_fn(self,features, labels, mode, params):


      
        return tf.estimator.EstimatorSpec(
            mode=mode, 
            loss=loss, 
            train_op=train_op) 
      
        
    def data_fn(self):
    
        self.trainX =
        self.trainY =



SyntaxError: invalid syntax (<ipython-input-1-a1db1e393c62>, line 8)

ML modeler를 이용하여 개발되는 Tensorflow 코드는 위와 같은 tensorflow estimator형식을 기반으로 작성되어야 합니다.   
Tensorflow Estimator에 대한 자세한 가이드는 아래의 링크에서 확인하실 수 있습니다.    
https://www.tensorflow.org/guide/estimators    

기본적으로 'Model'이라는 이름의 클래스에 다음과 같은 메서드를 포함합니다.   
* init : 필요한 기본 파라미터들을 정의하는 함수   
* model_fn : 모델 정의 함수   
* data_fn : 학습과 평가에 사용할 데이터를 정의하는 함수

또한, Model 클래스 외부에 다음의 2개의 기능 함수를 사용하실 수 있습니다.
* preprocessing : 배포 서버에 사용할 전처리 함수 (선택 사항)
* post_processing : 배포 서버에서 출력되는 output을 수정하는 함수 (선택 사항)         
    
   
   
      
이하의 guide는 위의 함수들의 작성 순서대로 안내됩니다. 

### 1. init
```python
def __init__(self,params):
    self.INPUT_TENSOR_NAME = "x"
    self.OUTPUT_TENSOR_NAME = "y"
    self.SIGNATURE_NAME = "serving_default"
    self.lr = params["learning_rate"]
    self.batch_size = params["batch_size"]
    self.graph_params = ["loss", "accuracy"]
```

init에는 아래의 6항목이 반드시 포함되어야 합니다.
```python
self.SIGNATURE_NAME     
self.INPUT_TENSOR_NAME    
self.OUTPUT_TENSOR_NAME    
self.lr    
self.batch_size    
self.graph_params    
```

* self.SIGNATURE_NAME :  학습이 완료된 모델을 배포할 때 사용됩니다. 변경 없이 "serving_default"로 사용하시면 됩니다.   


* self.INPUT_TENSOR_NAME은 모델에 사용할 input에 붙여지는 이름입니다.       
    여러 개의 input을 갖는 모델의 경우 INPUT_TENSOR_NAME과 data_fn의 self.trainX, self.evalX를 list 형태로 작성하시면 됩니다.   
        예) self.INPUT_TENSOR_NAME = ["x1", "x2", "x3"]       
            self.trainX = [trainx1, trainx2, trainx3]
            self.evalX = [evalX1, evalX2, evalX3]     
    
    
* self.OUTPUT_TENSOR_NAME은 모델의 output에 붙여지는 이름입니다.


* self.lr은 모델에 사용할 러닝레이트입니다. 학습시 UI에서 값을 입력받아 사용하고 싶은 경우 위처럼 params 딕셔너리에 사용하시면 됩니다.   
        ※  params는 UI에서 값을 변경해가며 사용하고 싶은 경우에 사용하는 딕셔너리입니다.   
            learning_rate 값을 self.lr = 0.001과 같이 바로 작성하지 않고 self.lr = params["learning_rate"]로 사용하시면   
            후에 UI에서 학습을 생성할 때 learning_rate에 사용할 값을 입력할 수 있게 됩니다.   
            UI에서 값을 변경해가며 입력하고 싶은 파라미터가 있는 경우 init에서 self.xxx = params["xxx"] 등으로 선언하신 후   
            model_fn에서 self.xxx로 사용하시면 됩니다.   


* self.batch_size는 학습에 적용할 배치사이즈 입니다. 러닝레이트와 마찬가지로 params를 사용하면 UI에서 값을 입력할 수 있습니다.   


* self.graph_params는 UI의 학습 상세 화면에서 그래프로 값을 보고 싶은 텐서들의 리스트입니다.       
    model_fn에 loss와 accuracy가 정의되어 있고 학습 진행 상황에 따라 그 값을 그래프로 보고 싶은 경우    
    예시처럼 ["loss", "accuracy"]로 기입하면 에폭마다 해당 값을 UI화면상의 그래프에 그려줍니다.  
    ※ 주의 : model_fn 내에 존재하는 텐서명을 입력하셔야 합니다.

### 2. model_fn
```python
def model_fn(self,features, labels, mode, params):   
    first_hidden_layer = tf.layers.dense(tf.cast(features[self.INPUT_TENSOR_NAME], dtype=tf.float32), 50, activation=tf.nn.relu)
    second_hidden_layer = tf.layers.dense(first_hidden_layer, 100, activation=tf.nn.relu)
    last_hidden_layer = tf.layers.dense(second_hidden_layer, 50, activation=tf.nn.relu)
    output_layer = tf.layers.dense(last_hidden_layer, 10)
    predictions = tf.argmax(output_layer, 1)


    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions={self.OUTPUT_TENSOR_NAME: predictions},
            export_outputs={self.SIGNATURE_NAME: PredictOutput({self.OUTPUT_TENSOR_NAME: predictions})}
        )

    y_label = tf.argmax(labels, 1)

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=output_layer))
    optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
    train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())

    eval_metric_ops={}
    accuracy = tf.metrics.accuracy(y_label, predictions)
    eval_metric_ops["accuracy"] = accuracy

    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops) 

```

model_fn의 인자 features는 {INPUT_TENSOR_NAME : trainX의 shape과 dtype에 맞는 텐서}로 구성된 딕셔너리입니다.   

모델 코드를 작성할 때 따로 input의 placeholder를 만들 필요 없이 위의 예시처럼 features를 사용하시면 됩니다.  

predict mode에서는 labels 인자가 존재하지 않기 때문에 if mode == tf.estimator.ModeKeys.PREDICT:은     
loss나 accuracy처럼 labels가 사용되는 텐서들보다 상위에 선언되어야 합니다.    

모델 코드를 모두 작성하신 후에 
tf.estimator.EstimatorSpec을 리턴하시면 됩니다. 
EstimatorSpec의 인자들은 각각 다음을 정의합니다.
* mode : 입력 인자에서 받았던 ModeKeys (그대로 mode 로 쓰시면 됩니다)
* loss : loss function을 이용하여 정의한 텐서
* train_op : minimized된 optimizer
* eval_metric_ops : 모형의 evaluation을 위한 metric을 정의하여 dictionary 형태로 추가(accuracy, RMSE 등)      


※ 주의 : optimizer는 반드시 optimizer라는 이름으로 예시처럼 온전하게 따로 선언해주어야 합니다.         
    - 아래와 같은 사용 불가능
```python
        train_op = tf.contrib.layers.optimize_loss(
           loss=loss,
           optimizer="Adam",
           learning_rate=self.lr,
           global_step=tf.train.get_global_step())
```

### 3. data_fn  
```python
def data_fn(self):
    data_path = "/mnt/project/demo/data/mnist-csv-data/data"
    trainX = np.loadtxt(data_path+"/trainX.csv")
    trainY = np.loadtxt(data_path+"/trainY.csv")
    evalX = np.loadtxt(data_path+"/evalX.csv")
    evalY = np.loadtxt(data_path+"/evalY.csv")
    self.trainX = trainX
    self.trainY = trainY
    self.evalX = evalX
    self.evalY = evalY
```

data_fn은 train과 eval에 사용할 데이터를 정의하는 함수입니다.   
모델에 사용할 수 있도록 데이터를 로드, 정제, train 데이터와 eval 데이터로 나누는 등의 처리를 한 뒤    
self.trainX, self.trainY, self.evalX, self.evalY에 각각의 데이터를 할당하시면 됩니다.    

     self.trainX : 학습 데이터    
     self.trainY : 학습 데이터의 레이블    
     self.evalX : 검증 데이터    
     self.evalY : 검증 데이터의 레이블    
        
학습만 하고자 하는 경우 self.evalX와 self.evalY는 작성하지 않으셔도 됩니다. 

### 4. 기타 추가 함수들 (preprocessing, post_processing)
```python
def preprocessing(image):
    import PIL.Image as pilimg
    img = pilimg.open(image)
    if img.size != (28,28):
        img = img.resize((28,28))
    if len(np.array(img).shape) == 3:
        img = img.convert('L')
    img = np.array(img).flatten()
    img = np.array([img])
    return img
       
        
def post_processing(output):
    if output in [0,2,4,6,8]:
        result = "짝수"
    elif output in [1,3,5,7,9]:
        result = "홀수"
    return result
   
```

* preprocessing 함수는 학습을 완료하고 해당 모델을 배포하는 경우에      
    배포 서버에 전송한 데이터를 모델에 통과시키기 전에 전처리하는 함수입니다.      
    예를 들어, MNIST 학습모델을 배포하고 배포 서버를 이용해 예측을 실행하고자 할 때,  
    .npy의 넘파이 배열 데이터를 전송하는 것이 아니라 필기체 숫자 이미지 데이터를 전송하고 싶은 경우      
    모델의 input은 해당 모델의 인풋 텐서에 맞는 형태의 배열 데이터만 받을 수 있기 때문에     
    이미지를 해당 인풋 텐서에 맞는 numpy로 변환해주는 작업이 필요합니다.    
    이 경우 이미지 파일을 로드하여 해당 인풋 텐서 형태의 numpy로 변환 후 return 해주는 함수를 작성하시면 됩니다.   
    (배포 서버로 전송한 파일의 경로가 함수의 인자로 사용됨, 배포 서버에서 받고자 하는 파일에 맞는 전처리 함수 작성)   
    
    
* post_processing 함수는 배포 서버에서 예측한 값의 출력을 처리하는 함수입니다.    
    위의 MNIST 예시의 경우 학습된 모델로 예측을 실행하면 0~9까지의 INT형 output이 나오게 되는데,    
    사용자가 원하는 조건대로 output을 재정의해서 리턴합니다.    
   
   

### 5. 주피터 노트북 상에서의 실행  
작성한 모델을 실행해보고 싶은 경우 if __name__ == "__main__": 의 하위에 아래처럼 실행문을 작성하시면 됩니다.      
첫 줄에 자신의 모델의 init에 사용한 params에 맞게 params 딕셔너리를 생성하고     
제일 아래의 steps를 입력하시고 나머지 코드는 그대로 사용하시면 됩니다.  


```python
if __name__ == "__main__":

    params = {"learning_rate":0.001, "batch_size":100}

    user_model = Model(params)
    user_model.data_fn()
    model = tf.estimator.Estimator(model_fn=user_model.model_fn, model_dir=None, params=params)
    train_dic = {}
    if type(user_model.INPUT_TENSOR_NAME) != list:
        train_dic[user_model.INPUT_TENSOR_NAME] = user_model.trainX
    else:
        for i in range(len(user_model.INPUT_TENSOR_NAME)):
            train_dic[user_model.INPUT_TENSOR_NAME[i]] = user_model.trainX[i]
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
            x=train_dic,
            y=user_model.trainY,
            batch_size=user_model.batch_size,
            num_epochs=None,
            shuffle=True)  
    model.train(input_fn=lambda : train_input_fn(), steps=100)
```